In [4]:
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import folium
import branca
import os

In [2]:
#get data from ref
eng = mariadb_connection()
df = pd.read_sql(
    """SELECT p.windturbine_id, windspeed, power, windfarm_id,latitude,longitude
        FROM mariadb_itw.windturbines as wt inner join mariadb_itw.powercurves as p
        on p.windturbine_id = wt.windturbine_id ;""",
    con=eng
    )




In [14]:
if not os.path.exists('figures'):
   os.makedirs('figures')

In [21]:

figs = {} # Create an empty figure to which we will add all the plotly figures
turbines_list=[] #Create an empty list to which we will append all the cities
html_list=[] #Create an empty list to which we will append all the exported html files
turbines = df['windturbine_id'].unique() # Get the list of city names
turbines=turbines[0:10]
for i, turb in enumerate(turbines, start = 0):
    turbines_list.append(turb)
    html_list.append('fig'+str(i)+'.html')
    df_turb = df[df['windturbine_id']==turb]

    fig=make_subplots(specs=[[{"secondary_y":True}]])

    fig.add_trace(                           #Add a bar chart to the figure
            go.Bar(
            x=df_turb['windspeed'],
            y=df_turb['power']                    
            ),
            secondary_y=False)               #The bar chart uses the primary y-axis (left)

    fig.update_layout(hoverlabel_bgcolor='#DAEEED',  #Change the background color of the tooltip to light gray
                title_text="Powercurves for turbine_id: "+ turb, #Add a chart title
                title_font_family="Times New Roman",
                title_font_size = 20,
                title_font_color="darkblue", #Specify font color of the title
                title_x=0.5, #Specify the title position
                
                yaxis_title='power',
                xaxis_title="windspeed")


    
    figs['fig'+str(i)] = fig
    fig.write_html('fig'+str(i)+".html")

df1=pd.DataFrame(turbines_list,columns =['windturbine_id'])
df2=pd.DataFrame(html_list,columns =['html_file'])
df3=pd.concat([df1, df2], axis=1)
df_geo=df[['windfarm_id','windturbine_id','latitude','longitude']].drop_duplicates('windturbine_id')
df_final=df3.merge(df_geo, on='windturbine_id', how='left')
df_final

,windturbine_id,html_file,windfarm_id,latitude,longitude
0,1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e,fig0.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7136,4.44707
1,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,fig1.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7169,4.45140
2,1ec6d7ce-93fd-6272-a3af-53533bbf8f7e,fig2.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7193,4.45693
3,1ec6d7ce-93ff-639c-9afd-53533bbf8f7e,fig3.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7212,4.46288
4,1ec6d7ce-9401-646c-b141-53533bbf8f7e,fig4.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7225,4.46990
5,1ec6d7ce-9403-651e-866d-53533bbf8f7e,fig5.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7243,4.47628
6,1ec6d7ce-9406-6688-98b1-53533bbf8f7e,fig6.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7262,4.48286
7,1ec6d7ce-9408-6802-b9c6-53533bbf8f7e,fig7.html,1ec6d7a1-4b96-67a4-9358-df2d2b033685,48.7300,4.48715
8,1ec6d7ce-940a-6b02-aa72-53533bbf8f7e,fig8.html,1ec6d7a1-4bf4-6ab6-a587-df2d2b033685,48.6630,4.13350
9,1ec6d7ce-940c-6b96-a952-53533bbf8f7e,fig9.html,1ec6d7a1-4bf4-6ab6-a587-df2d2b033685,48.6646,4.13952


In [25]:
#create list of colors to match with farms
colors= ["red", "blue", "green", "purple", "orange", "darkred","lightred", "beige", "darkblue", 
         "darkgreen", "cadetblue", "darkpurple", "white", "pink", 
         "lightblue", "lightgreen", "gray", "black", "lightgray"]

#create a number from O to number of farms_id
df_final['code'] = pd.factorize(df_final['windfarm_id'])[0] 
location = df_final['latitude'].mean(), df_final['longitude'].mean()
m = folium.Map(location=location,zoom_start=4)

for i,row in df_final.iterrows():

    html="""
    <iframe src=\"""" + df_final['html_file'][i] + """\" width="850" height="400"  frameborder="0">    
    """
    
    popup = folium.Popup(folium.Html(html, script=True))

    
    try:
        icon_color = colors[row['code']]
    except:
        #Catch nans
        icon_color = 'gray'

    folium.Marker([row['latitude'],row['longitude']],
                  popup=popup,icon=folium.Icon(icon='flag', prefix='fa')).add_to(m)
    
m

fig0.html
fig1.html
fig2.html
fig3.html
fig4.html
fig5.html
fig6.html
fig7.html
fig8.html
fig9.html
